#使用说明
1：新建一个workspace文件夹，将需要训练的data_dst.mp4和data_src.mp4复制进去。（如果自己有模型和视频分解文件请将model一并打包，也可以将本地的workspace整个打包）</br>
2：压缩这个workspace文件夹，压缩格式需为zip</br>
4：上传这个worksppace压缩包到你的谷歌云盘。</br>
5：执行配置环境。</br>
6：导入workspace.zip</br>
7：开始训练</br>
注意事项：如提示无法导入请查看左侧文件-drive/My Drive下是否有workspace.zip压缩包，如已上传却未查到请点击刷新或者稍等同步</br>
微信：wohaoshua2000</br>



# 1.环境准备

In [0]:
#查看显卡:若不是T4或P100请执行上方菜单：代码执行程序 -- 重置所有代码执行程序）重置后再次查看显卡，可多次执行！
! /opt/bin/nvidia-smi

In [0]:
#@title 配置环境

Mode = "1226" #@param ["clone", "1206", "1226"]

from pathlib import Path
if (Mode == "clone"):
  !git clone https://github.com/iperov/DeepFaceLab.git
elif Mode == "1206":
  !git clone https://github.com/iperov/DeepFaceLab.git
  %cd /content/DeepFaceLab
  !git checkout 154820a954dfbdfe43f291d3a7069a5e332d30c9
elif Mode == "1226":
  !git clone https://github.com/iperov/DeepFaceLab.git
  %cd /content/DeepFaceLab
  !git checkout 3849172a4384fc9670c3fd404e1c0029a35fba5c

!pip install -r /content/DeepFaceLab/requirements-colab.txt
!pip install --upgrade scikit-image
!apt-get install cuda-10-0
!pip install torch===1.3.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchvision===0.4.0 -f https://download.pytorch.org/whl/torch_stable.html
print("Done!")

Clone为20191226版，也可手动选择其他版本进行配置。




#2.导入素材

In [0]:
#@title 从云盘导入（导入你云盘的workspace.zip）

Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "model"]
Archive_name = "workspace.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  unzip_cmd=" -o -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "model":
  zip_and_copy("/content/workspace", "model")
print("Done!")



# 3.分解素材


In [0]:
#@title 分解视频（分解dst或src视频，分别提取）
Video = "data_src" #@param ["data_src", "data_dst"]

%cd "/content"

cmd = "DeepFaceLab/main.py videoed extract-video"

if Video == "data_dst":
  cmd+= " --input-file workspace/data_dst.* --output-dir workspace/data_dst/"
else:
  cmd+= " --input-file workspace/data_src.* --output-dir workspace/data_src/"
  
!python $cmd

In [0]:
#@title 提取脸部（提取dst或src视频，分别提取）
Data = "data_src" #@param ["data_src", "data_dst"]
Detector = "S3FD" #@param ["S3FD", "S3FD (Mark only)", "S3FD (Extract unaligned)", "MT"]
Debug = False #@param {type:"boolean"}

detect_type = "s3fd"
facetype = None

if Detector == "S3FD":
  detect_type = "s3fd"
elif Detector == "S3FD (Mark only)":
  detect_type = "s3fd"
  facetype = "mark_only"
elif Detector == "S3FD (Extract unaligned)":
  detect_type = "s3fd"
  facetype = "full_face_no_align"
elif Detector == "MT":
  detect_type = "mt"

folder = "workspace/"+Data
folder_align = folder+"/aligned"
debug_folder = folder_align+"/debug"

cmd = "DeepFaceLab/main.py extract --input-dir "+folder+" --output-dir "+folder_align

if Debug:
  cmd+= " --debug-dir "+debug_folder

cmd+=" --detector "+detect_type
  
if facetype:
  cmd+=" --face-type "+facetype
  
%cd "/content"
!python $cmd

In [0]:
#@title 脸图排序（排序后可导出后筛选）
Data = "data_src" #@param ["data_src", "data_dst"]
sort_type = "hist" #@param ["absdiff","hist", "hist-dissim", "face-yaw", "face-pitch", "vggface", "blur", "final", "final-no-blur"]

cmd = "DeepFaceLab/main.py sort --input-dir workspace/"+Data+"/aligned --by "+sort_type

%cd "/content"
!python $cmd

# 4.模型训练

In [0]:
#@title 继续训练（哪个模型就选哪个）
Model = "SAEHD" #@param ["SAE","SAEHD", "H128", "LIAEF128", "DF", "DF256", "TrueFace", "AVATAR", "DEV_FANSEG"]
Backup_every_hour = True #@param {type:"boolean"}

%cd "/content"

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive')

import psutil, os, time

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()

if (Backup_every_hour):
  if not os.path.exists('workspace.zip'):
    print("Creating workspace archive ...")
    !zip -r -q workspace.zip workspace
    print("Archive created!")
  else:
    print("Archive exist!")

if (Backup_every_hour):
  print("Time to end session: "+str(round((86400-uptime)/3600))+" hours")
  backup_time = str(3480)
  backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -1 -r -q workspace.zip workspace/model'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
elif (round(78200-uptime) > 0):
  print("Time to backup: "+str(round((78200-uptime)/900))+" hours")
  backup_time = str(round(78200-uptime))
  backup_cmd = " --execute-program "+backup_time+" \"import os; os.system('zip -1 -r -q workspace.zip workspace'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
else:
  print("Session expires in less than an hour.")
  backup_cmd = ""
    
cmd = "DeepFaceLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain/aligned --model-dir workspace/model --model "+Model
  
if (backup_cmd != ""):
  train_cmd = (cmd+backup_cmd)
else:
  train_cmd = (cmd)

!python $train_cmd

# 5.合成素材

In [0]:
#@title 合成图片（训练哪个就选哪个）
Model = "SAEHD" #@param ["SAEHD", "SAE", "Quick96", "H128", "LIAEF128", "DF", "AVATAR" ]

if Model == "AVATAR":  
  cmd = "DeepFaceLab/main.py convert --input-dir workspace/data_dst/aligned --output-dir workspace/data_dst/merged --model-dir workspace/model --model "+Model
else:
  cmd = "DeepFaceLab/main.py convert --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

%cd "/content"
!python $cmd

In [0]:
#@title 导出视频（导出后在云盘result.mp4 也可以左侧点文件-workspace里面result.mp4右键下载） 
!python DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4
!cp /content/workspace/result.mp4 /content/drive/My\ Drive/

# 压缩功能
压缩谷歌云盘的现有文件方便调用</br>举例：压缩谷歌云盘中的workspace文件夹，比如你的workspace文件夹在云盘根目录（如不在请在云盘移动到根目录）
</br>将压缩好的文件导入即可，用此办法以一反三，压缩不同的原有文件。

In [0]:
%cd /content/drive/My Drive/
!zip -r  workspace.zip  workspace

如果压缩其他文件夹，比如压缩model文件夹，假如model文件夹在workspace文件夹下面，那么可以将上面代码改成如下(请在上面更改)：</br>
%cd /content/drive/My Drive/workspace</br>
!zip -r  model.zip  model